# Tratamento dos dados brutos

Aqui a ideia é realizar a limpeza na base bruta que foi puxada no Kaggle. Aqui ocorre a remoção de linhas duplicadas e valores irrelevantes. Além disso, também tem que ser realizada a formatação de algumas informações para que a interpretação fique mais coesa nas próximas etapas.

In [1]:
import pandas as pd

df = pd.read_parquet('../data/bronze/salaries.parquet')

In [2]:
linhas_antes = df.shape[0]
df = df.drop_duplicates()
linhas_depois = df.shape[0]

print(f"Linhas removidas (duplicatas): {linhas_antes - linhas_depois}")

Linhas removidas (duplicatas): 39124


O primeiro ponto feito aqui foi a remoção de resultados duplicados, o que pode acabar viciando o modelo. Mesmo que a base é organizada, ainda sim aparecem resultados de Senior, por exemplo, que acabam se duplicando (talvez a mesma pessoa foi pesquisada ao longo de anos). Então isso tem que ser removido para não prejudicar o modelo posteriormente.

In [3]:
colunas_irrelevantes = ['salary', 'salary_currency']
df = df.drop(columns=colunas_irrelevantes)

Nessa análise de valores, não faz sentido utilizar o valor da moeda local do indivíduo, visto que a comparação e predição vão considerar o valore em dólar. Sendo assim, remover essas colunas.

In [4]:
mapa_experiencia = {
    'SE': 'Senior',
    'MI': 'Mid-level',
    'EN': 'Entry-level',
    'EX': 'Executive'
}
mapa_empresa = {
    'S': 'Small',
    'M': 'Medium',
    'L': 'Large'
}
mapa_emprego = {
    'FT': 'Full-time',
    'PT': 'Part-time',
    'CT': 'Contract',
    'FL': 'Freelance'
}

df['experience_level'] = df['experience_level'].map(mapa_experiencia).fillna(df['experience_level'])
df['company_size'] = df['company_size'].map(mapa_empresa).fillna(df['company_size'])
df['employment_type'] = df['employment_type'].map(mapa_emprego).fillna(df['employment_type'])

O *dataset* possui várias abreviações para poupar espaço, contudo, em uma análise dedicada, isso pode acabar ficando confuso. Sendo assim, substituir as siglas por legendas, para que fique mais claro o conteúdo analisado.

In [5]:
df_before = df.copy()

def filtra_outliers_por_experiencia(grupo, col_alvo="salary_in_usd", quantile_trim=0.95, fator_limite=1.2):
    salarios = grupo[col_alvo]

    salarios_trim = salarios[salarios <= salarios.quantile(quantile_trim)]
    media_base = salarios_trim.mean()

    limite_superior = media_base * fator_limite

    grupo_filtrado = grupo[salarios <= limite_superior]
    return grupo_filtrado

df = (
    df.groupby("experience_level", group_keys=False)
      .apply(filtra_outliers_por_experiencia)
      .reset_index(drop=True)
)

print(f"Linhas antes do filtro de outliers: {len(df_before)}")
print(f"Linhas depois do filtro de outliers: {len(df)}")

Linhas antes do filtro de outliers: 34024
Linhas depois do filtro de outliers: 22789


/tmp/ipykernel_30381/106553186.py:16: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(filtra_outliers_por_experiencia)


O dataset original apresentava diversos outliers de salário, especialmente em níveis de experiência júnior (por exemplo, registros de entry-level com salários próximos a 800 mil dólares), o que distorcia as métricas e prejudicava o ajuste do modelo. Para mitigar esse impacto, aplicamos um filtro de outliers por nível de experiência em duas etapas: primeiro, calculamos a média apenas sobre os 95% salários mais baixos de cada grupo (removendo temporariamente a cauda superior extrema) e, em seguida, eliminamos definitivamente os registros cujo salário ultrapassava 120% dessa média “podada”. Dessa forma, conseguimos remover valores claramente irreais ou atípicos sem descartar de forma agressiva casos altos, porém plausíveis.

In [7]:
df.to_parquet('../data/silver/salaries_cleaned.parquet', index=False)
display(df.head())

,work_year,experience_level,employment_type,job_title,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2025,Entry-level,Full-time,Data Scientist,62484,GB,0,GB,Medium
1,2025,Entry-level,Full-time,Data Scientist,52054,GB,0,GB,Medium
2,2025,Entry-level,Full-time,Data Scientist,68700,CA,0,CA,Medium
3,2025,Entry-level,Full-time,AI Engineer,83203,US,0,US,Large
4,2025,Entry-level,Full-time,Data Analyst,90000,US,0,US,Medium
